In [4]:
data = [
    {
        "question": "What was the largest dinosaur?",
        "answers": ["Argentinosaurus"],
        "positive_ctxs": [
            {
                "title": "Argentinosaurus",
                "text": "Argentinosaurus is a genus of titanosaur sauropod dinosaur first discovered by Guillermo Heredia in Argentina. The generic name refers to the country in which it was discovered. The dinosaur lived on the then-island continent of South America somewhere between 94 and 97 million years ago, during the Late Cretaceous Period. It is among the largest known dinosaurs."
            }
        ],
        "negative_ctxs": [
            {
                "title": "Tyrannosaurus",
                "text": "Tyrannosaurus is a genus of coelurosaurian theropod dinosaur. The species Tyrannosaurus rex (rex meaning 'king' in Latin), often called T. rex or colloquially T-Rex, is one of the most well-represented of the large theropods. Tyrannosaurus lived throughout what is now western North America, on what was then an island continent known as Laramidia."
            }
        ],
        "hard_negative_ctxs": [
            {
                "title": "Spinosaurus",
                "text": "Spinosaurus is a genus of theropod dinosaur that lived in what now is North Africa. Spinosaurus may be the largest of all known carnivorous dinosaurs, even larger than Tyrannosaurus and Giganotosaurus. Estimates published in 2005, 2007, and 2008 suggested that it was between 12.6–18 meters (41–59 ft) in length and 7 to 20.9 tonnes (7.7 to 23.0 short tons) in weight."
            }
        ]
    }
]

In [5]:
# import + don't forget to install
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer, DPRQuestionEncoder, DPRQuestionEncoderTokenizer
from torch.utils.data import DataLoader
import torch

# Initialize stuff
question_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained('facebook/dpr-question_encoder-single-nq-base')
question_model = DPRQuestionEncoder.from_pretrained('facebook/dpr-question_encoder-single-nq-base')

context_tokenizer = DPRContextEncoderTokenizer.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base')
context_model = DPRContextEncoder.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base')

# data is your list of dictionaries
questions = [item["question"] for i in data]
positive_contexts = [item["positive_ctxs"][0]["text"] for item in data]  # Just use the first positive context for each question

# Tokenize 
question_inputs = question_tokenizer(questions, return_tensors='pt', padding=True, truncation=True, max_length=512)
context_inputs = context_tokenizer(positive_contexts, return_tensors='pt', padding=True, truncation=True, max_length=512)

# Create DataLoader
batch_size = 16  # You can change this to w/e
dataset = torch.utils.data.TensorDataset(question_inputs['input_ids'], context_inputs['input_ids'])
dataloader = DataLoader(dataset, batch_size=batch_size)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizer'.


NameError: name 'item' is not defined

In [ ]:
for batch in dataloader:
    question_inputs, context_inputs = batch
    question_inputs = question_inputs.to(device) 
    context_inputs = context_inputs.to(device)  # Move the inputs to the Device

    # Forward pass through the models
    question_outputs = question_model(question_inputs)
    context_outputs = context_model(context_inputs)

    # Compute the loss, backwards pass and optimization, etc etc etc